This notebook describes how to create plots for the netCDF file(s) obtained after executing `DIVAnd`.     
We will use the [NCDatasets](https://github.com/Alexander-Barth/NCDatasets.jl) module (instead of [NetCDF](https://github.com/JuliaGeo/NetCDF.jl)) to read the result files and [PyPlot](https://github.com/JuliaPy/PyPlot.jl).

In [ ]:
using NCDatasets  
using PyPlot
if VERSION >= v"0.7.0-beta.0"

else
    using Compat: @info, @warn, @debug, @error, replace
    using Missings
end
using Compat

Adapt the file directory and name.      
We can use for example the salinity analysis produced by [15-example-analysis](./15-example-analysis.ipynb).

In [ ]:
datadir = "./data/"
datafile = joinpath(datadir, "Water_body_Salinity.4Danl.nc")
if isfile(datafile)
    @info("File already exists") 
else
    @error("File does not exist")
end

Open the dataset from the file for reading

In [ ]:
@info("Reading dataset")
ds = Dataset(datafile)

From the dataset we can extract the dimensions and variables of interest.

In [ ]:
lon = ds["lon"];
lat = ds["lat"];
depth = ds["depth"];
ttime = ds["time"];
salinity = ds["Salinity"];

Note that the time variable is an array of dates:

In [ ]:
ttime

# Create plots
## Simple pseudo color plot
Let's assume we want to plot a given time period at a given depth:

In [ ]:
depthplot = 20.;

First we extract the indices of the the time and depth arrays.

In [ ]:
timeindex = 4;
depthindex = findall(depth .== depthplot)[1];
@info("Time index: $timeindex")
@info("Depth index: $depthindex")

Then we extract the 2-D salinity field. If we are not sure about the dimension order, we can check the matrix size.

In [ ]:
@show size(salinity);
salinityplot = salinity[:, :, depthindex, timeindex];
@show size(salinityplot);
@show typeof(salinityplot);

### First plot
We use the `pcolormesh` (pseudo-color) function, after substituting the `missing`s by `NaN`s so that the land is masked.

In [ ]:
# replace(salinityplot, Missings.missing, NaN) ok with 1.0.0, not with 0.6.4
salinityplot[ismissing.(salinityplot)] .= NaN;
pcolormesh(lon, lat, permutedims(salinityplot, [2, 1]))
colorbar()
show()

## Contour plots
We explicitely specify the levels to be represented (from 15 to 25).    

In [ ]:
conts = contour(lon, lat, permutedims(salinityplot,[2,1]), levels=collect(15:.25:25))
clabel(conts, inline=1, fontsize=10);

## Filled contour plots
The aspect is somehow different from the pcolor.

In [ ]:
contourf(lon, lat, permutedims(salinityplot,[2,1]), levels=collect(15:1:25))

## Exercise
1. Load the results stored in `Water_body_Salinity.4Danl.nc` (obtained with notebook [15-example-analysis](./15-example-analysis.ipynb)).
2. Load the observation from `Salinity.bigfile` (downloaded in notebook [15-example-analysis](./15-example-analysis.ipynb)).
3. Display the salinity field at 50 meters (for the time period of your choice).
4. Add the observation value at that depth using the `scatter` function.

The solution is available in the notebook [16-plot-results-BlackSea](solutions/16-plot-results-BlackSea.ipynb) and should look like this:

<img src="./Images/BlackSea_plot.png" width="250px">